# Modelling the transmission grid in PyPSA-Eur

:::{note}
If you have not yet set up Python on your computer, you can execute this tutorial in your browser via [Google Colab](https://colab.research.google.com/). Click on the rocket in the top right corner and launch "Colab". If that doesn't work download the `.ipynb` file and import it in [Google Colab](https://colab.research.google.com/).

Then install the following packages by executing the following command in a Jupyter cell at the top of the notebook.

```sh
!pip install pypsa highspy
```
:::

## Background

### Motivation

* Conclusions drawn from energy system models only as good as underlying data and assumptions
* Modelling existing infrastructure is essential
* Geotagged, high-voltage electricity grid data not officially available to the public
* [ENTSO-E](https://www.entsoe.eu/data/map/) and some TSOs do provide public data in form of online maps or simplified datasets, however each come with own limitations (e.g. stylised geographic information), no open licence, potentially outdated
* Notable projects and datasets modelling the high-voltage grid in Europe include (alphabetical order):
  * [50Hertz Static Grid Model](#references)
  * [ELMOD](#references)
  * [ELMOD-DE](#references)
  * [Hutcheon & Bialek](#references)
  * [Jao Static Grid Model (no geospatial information)](#references)
  * [PyPSA-Eur (previously based on ENTSO-E online map)](#references)
  * [osmTGmod](#references)
  * [SciGrid (Power)](#references)

### Benefits of our implementation

* Reconstruction of the European high-voltage grid based on entirely public data
* Clear licensing Open Data Commons Open Database (ODbL 1.0)
* Always up to date and easy to maintain by accessing the OpenStreetMap Overpass turbo API
* Open (.csv) file format and standardised parameters, allows use outside of PyPSA-Eur
* Transparent, easy-to-reproduce workflow
* Compatibility with other PyPSA and PyPSA-Eur functions such as Dynamic Line Rating, enabling planned transmission projects (TYNDP, NEP)

## Reconstructing the European high-voltage grid from OpenStreetMap

The following section gives an overview of how a PyPSA-ready grid dataset is created from OpenStreetMap data. You can read the open-access **full paper** here:

Xiong, B., Fioriti, D., Neumann, F., Riepin, I., Brown, T. **Modelling the high-voltage grid using open data for Europe and beyond.** Sci Data 12, 277. https://doi.org/10.1038/s41597-025-04550-7 (2025). https://www.nature.com/articles/s41597-025-04550-7

### Overview

Map of the OSM-based European high-voltage grid. Map generated using prebuilt network published on [Zenodo](#references).

<img src="https://media.springernature.com/full/springer-static/image/art%3A10.1038%2Fs41597-025-04550-7/MediaObjects/41597_2025_4550_Fig6_HTML.png" width="500px" />

### Process diagram

**Process diagram** depicting the creation of the European high-voltage grid from OSM data, implemented through individual [Snakemake rules](#references).

<img src="https://media.springernature.com/full/springer-static/image/art%3A10.1038%2Fs41597-025-04550-7/MediaObjects/41597_2025_4550_Fig1_HTML.png" alt="Process diagram: Snakemake" width="700px" />

In step 3, A topologically and electrically network is then built using a sequence of steps.


<img src="https://media.springernature.com/full/springer-static/image/art%3A10.1038%2Fs41597-025-04550-7/MediaObjects/41597_2025_4550_Fig4_HTML.png" alt="Process diagram: Schematic" width="700px">

### Bus clustering
**Example of bus clustering.** The darkred shape represents the union of the buffer around virtual buses and an original OSM substation polygon (yellow). The bright red dot represents the internal point of the union, this point sets the geographic coordinates of the obtained bus. Lines and cables are connected to the respective voltage level within the substation, transformers are added to connect buses of different voltage levels.

<img src="https://media.springernature.com/full/springer-static/image/art%3A10.1038%2Fs41597-025-04550-7/MediaObjects/41597_2025_4550_Fig5_HTML.png" alt="Bus clustering" width="700px"/>

### Validation and metrics

**Comparing the OSM-based grid to ENTSO-map**.

<img src="https://media.springernature.com/full/springer-static/image/art%3A10.1038%2Fs41597-025-04550-7/MediaObjects/41597_2025_4550_Fig7_HTML.png" alt="Comparison of route and circuit lengths" width="700px"/>

**Weighted degree distribution** before and after clustering (NUTS2).

<img src="https://media.springernature.com/full/springer-static/image/art%3A10.1038%2Fs41597-025-04550-7/MediaObjects/41597_2025_4550_Fig9_HTML.png" alt="WEighted degree distribution" width="700px"/>

**Comparison with public, geocoded 50Hertz static grid model**.

<img src="https://media.springernature.com/full/springer-static/image/art%3A10.1038%2Fs41597-025-04550-7/MediaObjects/41597_2025_4550_Fig17_HTML.png" alt="Comparison with 50Hertz SGM" width="700px"/>

## Exploring the data (base network)

This is an interactive part of the workshop, you can follow the live demonstration or try out the notebook yourself.

First, we need to import `geopandas`, `folium`, and `pypsa`.

In [ ]:
import geopandas as gpd
import numpy as np
import pypsa

Let's import the base network created from the above workflow.

In [ ]:
network_path = "resources/base.nc"
n = pypsa.Network(network_path)

PyPSA calculates the line parameters just before the model is solved. If we want to see the underlying impedance $r$, reactance $x$, and susceptance $b$, we can trigger the calculation, manually.

In [ ]:
n.calculate_dependent_values()
n.lines["i_nom"] = (
    (n.lines.s_nom / n.lines.v_nom / n.lines.num_parallel)
    .div(np.sqrt(3))
    .round(3)
)  # kA

So how did this work? Underneath, PyPSA maps each line to a library of built-in standard line types to obtain per linetype, per km values

In [ ]:
print(n.line_types.head())
print("\nAC line types in the base network:")
print(f"AC Line types: {sorted(n.lines.type.unique())}")

We import a helper function to visualise the detailed network, interactively

In [ ]:
from resources.helpers import create_geometries
buses, lines, links, converters, transformers = create_geometries(n)

### Interactive map

Using the PyPSA base network, let's create an **interactive map**. To help visualise the underlying input data, we also import the cleaned substations

In [ ]:
stations_polygon = gpd.read_file("resources/stations_polygon.geojson")
buses_polygon = gpd.read_file("resources/buses_polygon.geojson")

Stacking everything together on a single folium map

In [ ]:
map = None
b_popup = True

map = stations_polygon.explore(color="yellow", popup=b_popup)
map = buses_polygon.query("dc == False").explore(color="red", popup=b_popup, m=map)
map = buses_polygon.query("dc == True").explore(color="purple", popup=b_popup, m=map)
map = lines.query("v_nom <= 230").explore(color="green", popup=b_popup, m=map)
map = lines.query("(v_nom > 230) & (v_nom <= 330)").explore(color="orange", popup=b_popup, m=map)
map = lines.query("(v_nom > 330) & (v_nom <= 420)").explore(color="darkred", popup=b_popup, m=map)
map = lines.query("v_nom > 420").explore(color="pink", popup=b_popup, m=map)
map = links.explore(color="purple", popup=b_popup, m=map)
map = converters.explore(color="black", popup=b_popup, m=map)
map = transformers.explore(color="pink", popup=b_popup, m=map)

How about locating the ["Kabeldiagonale"](https://www.50hertz.com/de/Netz/Netzausbau/ProjekteanLand/BerlinerProjekte/KabeldiagonaleBerlin/) in Berlin?

In [ ]:
map

**Questions**
1. What are its attributes ($i_{nom}^{max}$, $v_{nom}$, and $s_{nom}^{max}$)? 
2. How many circuits does it have?
3. How long is the longest section within Berlin?
4. What are the names of the two substations connected to this section?

## References

* 50Hertz. **Static grid model**. https://www.50hertz.com/Transparency/GridData/Gridfigures/Staticgridmodel/ (2022).
* Egerer, J. et al. **Electricity sector data for policy-relevant modeling: Data documentation and applications to the German and European electricity markets**. Research Report 72, DIW Data Documentation (2014).
* Egerer, J. **Open Source Electricity Model for Germany (ELMOD-DE)**. Data Documentation (2016).
* JAO. **Static Grid Model**. https://www.jao.eu/static-grid-model (2023).
* Hörsch, J., Hofmann, F., Schlachtberger, D. & Brown, T. **PyPSA-Eur: An open optimisation model of the European transmission system**. Energy Strategy Reviews 22, 207–215, https://doi.org/10.1016/j.esr.2018.08.012 (2018).
* Hutcheon, N. & Bialek, J. W. **Updated and validated power flow model of the main continental European transmission network**. In 2013 IEEE Grenoble Conference, 1–5, https://doi.org/10.1109/PTC.2013.6652178 (2013).
* Medjroubi, W., Müller, U. P., Scharf, M., Matke, C. & Kleinhans, D. **Open Data in Power Grid Modelling: New Approaches Towards Transparent Grid Models**. Energy Reports 3, 14–21, https://doi.org/10.1016/j.egyr.2016.12.001 (2017).
* Mölder, F. et al. **Sustainable data analysis with Snakemake**. https://doi.org/10.12688/f1000research.29032.2 (2021).
* **OSMTGmod Documentation** 0.1.0. https://github.com/wupperinst/osmTGmod/blob/master/osmTGmod_documentation_0.1.0.pdf (2017).
* Xiong, B., Fioriti, D., Neumann, F., Riepin, I., Brown, T. **Modelling the high-voltage grid using open data for Europe and beyond.** Sci Data 12, 277. https://doi.org/10.1038/s41597-025-04550-7 (2025).
* Xiong, B., Fioriti, D., Neumann, F., Riepin, I., Brown, T. **Prebuilt Electricity Network for PyPSA-Eur based on OpenStreetMap Data (0.6) [Data set].** Zenodo. https://doi.org/10.5281/zenodo.14144752 (2024).